compare calibration result and simulation result in the instant loop detector

In [55]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-02'
detector = 'e2w_in'
number = 2
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [56]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [57]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_out,1.577923e+09,enter,0_e2w_out,4.46,5.00,DEFAULT_VEHTYPE,None
3,e2w_out,1.577923e+09,enter,1_e2w_out,6.62,5.00,DEFAULT_VEHTYPE,17.69


In [58]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_out,1.577923e+09,4.998,0.6,1577923299,1577923253,4.998,0.0,-0.35,4.998
1,1_e2w_out,1.577923e+09,4.998,0.6,1577923317,1577923271,4.998,0.0,-0.35,4.998


In [59]:
print(len(data), len(df_instant_out))

2 2


compare time error and speed error in loop detector between simulation result and calibration result

In [60]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_out,1.577923e+09,4.46,1.577923e+09,4.998,1577923299,0.0
1,1_e2w_out,1.577923e+09,6.62,1.577923e+09,4.998,1577923317,0.0


In [61]:
#compare_df = compare_df.head(19)

In [62]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.34999990463256836)

In [63]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(5.804999947547913)

In [64]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(4.998)

In [65]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(5.54)

In [66]:
start = min(compare_df['time_sim_error'].min(),compare_df['time_calib_error'].min()) 
end = max(compare_df['time_sim_error'].max(), compare_df['time_calib_error'].max())


In [67]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [68]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

5.490000009536743 6.119999885559082
-0.34999990463256836 -0.34999990463256836


In [69]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

4.46 6.62
4.998 4.998


In [70]:
start = min(compare_df['speed_sim_error'].min(),compare_df['speed_calib_error'].min()) 
end = max(compare_df['speed_sim_error'].max(), compare_df['speed_calib_error'].max())

In [71]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [72]:
#import os
#os.environ["HORNSGATAN_HOME"]
